In [1]:
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def scrap_data(ticker,start,end):
    df = web.DataReader(ticker,'yahoo',start,end)
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)    
    return(df)

In [3]:
def boost(df):
    print('Boosting')
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    model = xgb.XGBRegressor( n_estimators=20000, learning_rate=0.09, gamma=1, subsample=0.1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    print('Boost ok')
    return(df)

In [34]:
def kerasReg(df):
    # Regression Example With Boston Dataset: Standardized and Larger
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.wrappers.scikit_learn import KerasRegressor
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import KFold
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    
    # define the model
    def larger_model():
        # create model
        model = Sequential()
        model.add(Dense(13, input_dim=6, kernel_initializer='normal', activation='relu'))
        model.add(Dense(6, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    
    print('Boosting')
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]
    
    model = larger_model()
    
    model.fit( Xtrain, ytrain,
    batch_size=1,
    epochs=50,
    verbose=1,
    callbacks=None,
    validation_split=0.1,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    print('Boost ok')
    
    # evaluate model with standardized dataset
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=1, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = KFold(n_splits=2)
    results = cross_val_score(pipeline, Xtrain, ytrain, cv=kfold)
    print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    
    return(df)
    
    
    
    
    

In [19]:
import catboost as ctb
def catboost(df):
    
    print('CAT Boosting')
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    model = ctb.CatBoostRegressor(iterations=1000, depth=12, loss_function="RMSE",verbose=False)#n_estimators=20000, learning_rate=0.09, gamma=1, subsample=0.1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    print('Boost ok')
    return(df)

In [5]:
print('')
print('Sraping tickers')
constituents = pd.read_csv('constituents_csv.csv')
print('Scrap -----> ok')
# PARAMETRES TEMPORELS INITIAUX
start = pd.to_datetime('2014-01-06')
end = pd.to_datetime('2018-12-31')

########################
#### MAIN SKAN55 #######
########################

for loop in range(50,51):    
#for loop in range(amorce,len(constituents)):
    print('Loop :',loop)
    
   # try:

    ticker = (constituents.iloc[loop]['Symbol'])
    name = constituents.iloc[loop]['Name']
    sector = constituents.iloc[loop]['Sector']

    print(' --- Loop :',loop,' --- ',end='')
    print(' --- Symbol : ',Fore.YELLOW,ticker,Style.RESET_ALL,' --- ',end='')
    print(' --- Name : ',name,' --- ',end='')
    print(' --- Sector : ',sector,' --- ')

    global delta,bloc1,bloc2
    tmps1=time.time()
    print('Scraping data...')

    df = scrap_data(ticker,start,end)

    tmps2=round(time.time()-tmps1,2)
    print ("executed in = %f" %tmps2,'seconds')

    print('\n')
    print('GOLEM begins Computing...')
    print('\n')
    delta = round(df.shape[0])
    bloc1 = round(delta*0.80)
    bloc2 = delta - bloc1
    print("# of periods : ",delta)
    print('On 80% - 20% slash : ')
    print('Bloc 1 : ',bloc1,'\nBloc 2 :',bloc2,' periods ')
    print('First period :',df.index[0])
    print('Split period :',df.index[bloc1-1])
    print('Last period :',df.index[df.shape[0]-1])
    print('\n')

    print('Data prep')


Sraping tickers
Scrap -----> ok
Loop : 50
 --- Loop : 50  ---  --- Symbol :   AIV   ---  --- Name :  Apartment Investment & Management  ---  --- Sector :  Real Estate  --- 
Scraping data...
executed in = 1.100000 seconds


GOLEM begins Computing...


# of periods :  1256
On 80% - 20% slash : 
Bloc 1 :  1005 
Bloc 2 : 251  periods 
First period : 2014-01-06 00:00:00
Split period : 2017-12-29 00:00:00
Last period : 2018-12-31 00:00:00


Data prep


In [6]:
df.tail()

,High,Low,Open,Volume,Close
Date,,,,,
2018-12-24,44.590000,42.209999,44.340000,981300.0,41.230518
2018-12-26,43.430000,42.090000,42.330002,2878400.0,42.412159
2018-12-27,43.540001,42.290001,43.160000,2468500.0,42.500050
2018-12-28,44.259998,43.090000,43.950001,1620700.0,42.460983
2018-12-31,43.889999,42.990002,43.520000,1530000.0,42.851612


In [7]:
bdf = boost(df)
bdf.tail()

Boosting
Boost ok


,High,Low,Open,Volume,Close,Close.S,Close.S2
Date,,,,,,,
2018-12-24,44.590000,42.209999,44.340000,981300.0,41.230518,39.784389,39.784389
2018-12-26,43.430000,42.090000,42.330002,2878400.0,42.412159,40.203186,40.203186
2018-12-27,43.540001,42.290001,43.160000,2468500.0,42.500050,40.082439,40.082439
2018-12-28,44.259998,43.090000,43.950001,1620700.0,42.460983,40.596268,40.596268
2018-12-31,43.889999,42.990002,43.520000,1530000.0,42.851612,40.540207,40.540207


In [10]:
cdf = catboost(df)
cdf.tail()

CAT Boosting
Boost ok


,High,Low,Open,Volume,Close,Close.S,Close.S2
Date,,,,,,,
2018-12-24,44.590000,42.209999,44.340000,981300.0,41.230518,37.481722,37.481722
2018-12-26,43.430000,42.090000,42.330002,2878400.0,42.412159,40.237829,40.237829
2018-12-27,43.540001,42.290001,43.160000,2468500.0,42.500050,40.562232,40.562232
2018-12-28,44.259998,43.090000,43.950001,1620700.0,42.460983,40.698974,40.698974
2018-12-31,43.889999,42.990002,43.520000,1530000.0,42.851612,40.574971,40.574971


In [35]:
kdf = kerasReg(df)
kdf.tail()

Boosting
Train on 1128 samples, validate on 126 samples
Epoch 1/50
1128/1128 [==============================] - 2s 2ms/step - loss: 1587.2409 - val_loss: 621.0731
Epoch 2/50
1128/1128 [==============================] - 1s 936us/step - loss: 461.9872 - val_loss: 409.5546
Epoch 3/50
1128/1128 [==============================] - 1s 937us/step - loss: 353.1233 - val_loss: 428.5080
Epoch 4/50
1128/1128 [==============================] - 1s 937us/step - loss: 320.5046 - val_loss: 419.9019
Epoch 5/50
1128/1128 [==============================] - 1s 935us/step - loss: 524.3421 - val_loss: 1638.4192
Epoch 6/50
1128/1128 [==============================] - 1s 934us/step - loss: 1085.1675 - val_loss: 1529.6324
Epoch 7/50
1128/1128 [==============================] - 1s 930us/step - loss: 1005.4789 - val_loss: 1438.5198
Epoch 8/50
1128/1128 [==============================] - 1s 963us/step - loss: 933.8168 - val_loss: 1351.4205
Epoch 9/50
1128/1128 [==============================] - 1s 942us/step - los

,High,Low,Open,Volume,Close,Close.S,Close.S2
Date,,,,,,,
2018-12-24,44.590000,42.209999,44.340000,981300.0,41.230518,35.347736,35.347736
2018-12-26,43.430000,42.090000,42.330002,2878400.0,42.412159,35.347736,35.347736
2018-12-27,43.540001,42.290001,43.160000,2468500.0,42.500050,35.347736,35.347736
2018-12-28,44.259998,43.090000,43.950001,1620700.0,42.460983,35.347736,35.347736
2018-12-31,43.889999,42.990002,43.520000,1530000.0,42.851612,35.347736,35.347736


In [ ]:
web.DataReader(ticker,'yahoo','2019-01-01','2019-01-09')